In [1]:
!python -m pip install pypdf faiss-cpu --quiet
!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
!python -m pip install langchain-openai --quiet
!python -m pip install langchain-community langchainhub langchain-chroma langchain langchain-experimental --quiet

In [ ]:
# # cell 1: Installation and Imports
# # Run this cell first to install required packages
# # !pip install langchain langchain-community langgraph chromadb sqlite3 pandas numpy matplotlib seaborn openai python-dotenv fastapi uvicorn python-multipart
# # !pip install azure-identity azure-keyvault-secrets

# import os
# import sqlite3
# import pandas as pd
# import numpy as np
# import chromadb
# from chromadb.config import Settings
# from langchain.schema import Document
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chat_models import AzureChatOpenAI
# from langchain.schema import HumanMessage, SystemMessage
# import json
# import re
# from typing import Dict, List, Any, Optional
# import matplotlib.pyplot as plt
# import seaborn as sns
# from datetime import datetime
# import warnings
# import sys
# from azure.identity import DefaultAzureCredential
# from azure.keyvault.secrets import SecretClient
# import os
# from dotenv import load_dotenv
# warnings.filterwarnings('ignore')
# load_dotenv()
# print("All packages imported successfully!")

All packages imported successfully!


In [3]:
# Cell 1: Import all required libraries
import sqlite3
import pandas as pd
import numpy as np
import os
from typing import Dict, List, Any, Optional, Tuple
import json
import math
from datetime import datetime, timedelta
import chromadb
from chromadb.config import Settings
import warnings
warnings.filterwarnings('ignore')

print(" Initializing Loan Navigator Agent Suite...")


 Initializing Loan Navigator Agent Suite...


In [4]:
# Cell 2: Environment Setup and Configuration
class Config:
    """Configuration class for environment variables and settings"""
    
    def __init__(self):
        # Azure OpenAI Configuration
        self.azure_endpoint = "https://eastus.api.cognitive.microsoft.com/"
        self.azure_api_key = "cbb318fdda184080a03be71c1c768bae"
        self.api_version = "2024-08-01-preview"
        self.deployment_name = "gpt4o"
        
        # Database paths
        self.db_path = "Loan_BlueLoan4all.sqlite"
        self.chroma_path = "./chroma_db"
        self.policy_docs_path = "./BL4A_policy_docs"
        
        # Agent settings
        self.max_retries = 3
        self.timeout = 30

config = Config()
print(" Configuration loaded successfully")


 Configuration loaded successfully


In [ ]:
# Cell 3: Database Manager with Enhanced Features
class DatabaseManager:
    """Enhanced database manager with query capabilities"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.connection = None
        self.connect()
    
    def connect(self):
        """Establish database connection"""
        try:
            self.connection = sqlite3.connect(self.db_path)
            print(f" Connected to database: {self.db_path}")
        except Exception as e:
            print(f" Database connection failed: {e}")
    
    def get_table_info(self):
        """Get all table names and their schemas"""
        cursor = self.connection.cursor()
        
        # Get all table names
        # Fetch all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(f"- {table[0]}")
        
        print(" DATABASE TABLES:")
        print("-" * 50)
        
        for table in tables:
            print(f"\n Table: {table[0]}")
            print("Columns:")
            
            # Get column information
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            
            for col in columns:
                col_id, col_name, col_type, not_null, default_val, pk = col
                print(f"  - {col_name} ({col_type})")
            
            # Show sample data
            print(f"\nSample data from {table}:")
            try:
                df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 3", self.connection)
                if not df.empty:
                    print(df.to_string(index=False))
                else:
                    print("  (No data)")
            except Exception as e:
                print(f"  Error reading data: {e}")
        
        return tables
    
    def execute_safe_query(self, query: str, params: tuple = None) -> pd.DataFrame:
        """Execute SQL query safely with parameterization"""
        try:
            if params:
                df = pd.read_sql_query(query, self.connection, params=params)
            else:
                df = pd.read_sql_query(query, self.connection)
            return df
        except Exception as e:
            print(f" Query execution failed: {e}")
            return pd.DataFrame()
    
    def get_loan_details(self, loan_id: str = None) -> pd.DataFrame:
        """Get loan details with optional filtering"""
        if loan_id:
            query = "SELECT * FROM loan_data WHERE loan_id = ?"
            return self.execute_safe_query(query, (loan_id,))
        else:
            query = "SELECT * FROM loan_data LIMIT 5"
            return self.execute_safe_query(query)
    
    def close(self):
        """Close database connection"""
        if self.connection:
            self.connection.close()
            print(" Database connection closed")

# Initialize database
db_manager = DatabaseManager(config.db_path)


 Connected to database: Loan_BlueLoan4all.sqlite


In [9]:
# Cell 4: Display Database Structure
print(" ANALYZING DATABASE STRUCTURE...")
print("=" * 60)
tables = db_manager.get_table_info()
print(tables)
print(f"\n Total tables found: {len(tables)}")


 ANALYZING DATABASE STRUCTURE...
Tables in the database:
 DATABASE TABLES:
--------------------------------------------------
[]

 Total tables found: 0


In [10]:
# Cell 5: Amortization Calculator Engine
class AmortizationCalculator:
    """What-If Calculator Agent for loan simulations"""
    
    def __init__(self):
        print(" Amortization Calculator initialized")
    
    def calculate_emi(self, principal: float, annual_rate: float, tenure_months: int) -> float:
        """Calculate EMI using standard formula"""
        monthly_rate = annual_rate / 12 / 100
        emi = (principal * monthly_rate * (1 + monthly_rate) ** tenure_months) / \
              ((1 + monthly_rate) ** tenure_months - 1)
        return round(emi, 2)
    
    def generate_amortization_schedule(self, principal: float, annual_rate: float, 
                                    tenure_months: int, prepayment: float = 0) -> Dict[str, Any]:
        """Generate complete amortization schedule"""
        monthly_rate = annual_rate / 12 / 100
        emi = self.calculate_emi(principal, annual_rate, tenure_months)
        
        schedule = []
        balance = principal
        total_interest = 0
        
        print(f"Generating amortization schedule...")
        print(f"   Loan Amount: ₹{principal:,.2f}")
        print(f"   Interest Rate: {annual_rate}%")
        print(f"   Tenure: {tenure_months} months")
        print(f"   EMI: ₹{emi:,.2f}")
        
        for month in range(1, tenure_months + 1):
            interest_component = balance * monthly_rate
            principal_component = emi - interest_component
            
            # Apply prepayment in first month if specified
            if month == 1 and prepayment > 0:
                principal_component += prepayment
                print(f"   Prepayment applied: ₹{prepayment:,.2f}")
            
            balance -= principal_component
            total_interest += interest_component
            
            schedule.append({
                'month': month,
                'emi': emi,
                'principal': round(principal_component, 2),
                'interest': round(interest_component, 2),
                'balance': round(max(balance, 0), 2)
            })
            
            if balance <= 0:
                break
        
        # Calculate savings with prepayment
        original_total = emi * tenure_months
        new_total = sum(entry['emi'] for entry in schedule)
        interest_saved = total_interest - (original_total - principal)
        
        result = {
            'original_emi': emi,
            'amortization_schedule': schedule,
            'total_interest_paid': round(total_interest, 2),
            'total_payment': round(sum(entry['emi'] for entry in schedule), 2),
            'interest_saved': round(interest_saved, 2),
            'months_saved': tenure_months - len(schedule)
        }
        
        return result
    
    def simulate_prepayment(self, loan_data: Dict, prepayment_amount: float) -> Dict[str, Any]:
        """Simulate prepayment scenario"""
        print(f" SIMULATING PREPAYMENT: ₹{prepayment_amount:,.2f}")
        
        principal = loan_data['loan_amount']
        interest_rate = loan_data['interest_rate']
        remaining_months = loan_data['tenure_months']
        amount_paid = loan_data.get('amount_paid', 0)
        
        # Calculate remaining principal
        emi_paid = amount_paid / loan_data['monthly_emi'] if loan_data['monthly_emi'] > 0 else 0
        remaining_principal = principal - (emi_paid * loan_data['monthly_emi'])
        
        if remaining_principal <= 0:
            return {"error": "Loan already paid off"}
        
        # Generate scenarios
        original_schedule = self.generate_amortization_schedule(
            remaining_principal, interest_rate, remaining_months
        )
        
        new_schedule = self.generate_amortization_schedule(
            remaining_principal, interest_rate, remaining_months, prepayment_amount
        )
        
        comparison = {
            'original': original_schedule,
            'with_prepayment': new_schedule,
            'prepayment_impact': {
                'emi_reduction': original_schedule['original_emi'] - new_schedule.get('original_emi', 0),
                'interest_saved': new_schedule['interest_saved'],
                'months_saved': new_schedule['months_saved']
            }
        }
        
        return comparison

# Initialize calculator
calculator = AmortizationCalculator()


 Amortization Calculator initialized


In [11]:
# Cell 6: Policy Guru Agent (RAG-based)
class PolicyGuruAgent:
    """Policy Guru Agent for document retrieval and compliance"""
    
    def __init__(self, chroma_path: str):
        self.chroma_path = chroma_path
        self.client = None
        self.collection = None
        self.initialize_chroma()
    
    def initialize_chroma(self):
        """Initialize ChromaDB connection"""
        try:
            self.client = chromadb.PersistentClient(path=self.chroma_path)
            self.collection = self.client.get_collection("policy_documents")
            print(" Policy Guru Agent initialized - ChromaDB connected")
        except Exception as e:
            print(f" ChromaDB initialization failed: {e}")
            self.setup_fallback_policies()
    
    def setup_fallback_policies(self):
        """Setup fallback policy knowledge base"""
        self.fallback_policies = {
            "prepayment": {
                "policy": "Prepayment up to 25% of outstanding principal allowed annually without charges",
                "conditions": ["Minimum prepayment: ₹5,000", "No charges for prepayment", "Can reduce EMI or tenure"],
                "source": "BlueLoans4all_Loan_Policy_Manual"
            },
            "topup_eligibility": {
                "policy": "Top-up loans available after 6 months of timely payments",
                "conditions": ["Minimum 6 months repayment history", "No defaults in last 3 months", "Credit score > 650"],
                "source": "BlueLoans4all_Topup_and_Upgrade_Policy"
            },
            "eligibility": {
                "policy": "Basic eligibility: Age 21-60, Minimum income ₹25,000/month, Indian resident",
                "conditions": ["Valid ID proof required", "Income documentation", "Bank statements for 3 months"],
                "source": "BlueLoans4all_Risk_and_Underwriting_Policy"
            }
        }
        print("  Using fallback policy knowledge base")
    
    def query_policy(self, question: str, top_k: int = 3) -> Dict[str, Any]:
        """Query policy documents using RAG"""
        print(f" Policy Guru searching for: '{question}'")
        
        try:
            # ChromaDB query
            results = self.collection.query(
                query_texts=[question],
                n_results=top_k
            )
            
            documents = results['documents'][0] if results['documents'] else []
            metadatas = results['metadatas'][0] if results['metadatas'] else []
            distances = results['distances'][0] if results['distances'] else []
            
            response = {
                "question": question,
                "sources": [],
                "answer": "",
                "confidence": "high" if documents else "medium"
            }
            
            if documents:
                for i, doc in enumerate(documents):
                    source = metadatas[i].get('source', 'Unknown') if i < len(metadatas) else 'Unknown'
                    response["sources"].append({
                        "content": doc[:200] + "..." if len(doc) > 200 else doc,
                        "source": source,
                        "similarity": 1 - distances[i] if i < len(distances) else 0
                    })
                
                # Simple answer generation from top document
                response["answer"] = self.generate_answer(question, documents[0])
            else:
                # Fallback to predefined policies
                response = self.fallback_query(question)
                response["confidence"] = "medium"
            
            return response
            
        except Exception as e:
            print(f" Policy query failed: {e}")
            return self.fallback_query(question)
    
    def fallback_query(self, question: str) -> Dict[str, Any]:
        """Fallback query using predefined policies"""
        question_lower = question.lower()
        
        if any(word in question_lower for word in ['prepay', 'prepayment', 'early']):
            policy_data = self.fallback_policies["prepayment"]
        elif any(word in question_lower for word in ['topup', 'top-up', 'additional loan']):
            policy_data = self.fallback_policies["topup_eligibility"]
        elif any(word in question_lower for word in ['eligible', 'eligibility', 'qualify']):
            policy_data = self.fallback_policies["eligibility"]
        else:
            policy_data = {
                "policy": "For specific policy queries, please contact customer support",
                "conditions": [],
                "source": "General Policy"
            }
        
        return {
            "question": question,
            "sources": [{
                "content": policy_data["policy"],
                "source": policy_data["source"],
                "similarity": 0.7
            }],
            "answer": f"Based on {policy_data['source']}: {policy_data['policy']}",
            "confidence": "medium"
        }
    
    def generate_answer(self, question: str, context: str) -> str:
        """Generate answer from context (simplified)"""
        # In production, this would use Azure OpenAI
        return f"Based on our policy documents: {context[:150]}..."

# Initialize Policy Guru
policy_guru = PolicyGuruAgent(config.chroma_path)


 ChromaDB initialization failed: Collection [policy_documents] does not exists
  Using fallback policy knowledge base


In [12]:
# Cell 7: SQL Analyst Agent
class SQLAnalystAgent:
    """SQL Analyst Agent for database queries"""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager
        self.schema_info = self.get_schema_info()
        print(" SQL Analyst Agent initialized")
    
    def get_schema_info(self) -> str:
        """Get database schema information"""
        try:
            cursor = self.db.connection.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            
            schema_info = "Database Schema:\n"
            for table in tables:
                table_name = table[0]
                cursor.execute(f"PRAGMA table_info({table_name})")
                columns = cursor.fetchall()
                schema_info += f"\nTable: {table_name}\n"
                for col in columns:
                    schema_info += f"  - {col[1]} ({col[2]})\n"
            
            return schema_info
        except Exception as e:
            return f"Error reading schema: {e}"
    
    def natural_language_to_sql(self, question: str) -> str:
        """Convert natural language to SQL (simplified)"""
        question_lower = question.lower()
        
        # Simple intent recognition
        if any(word in question_lower for word in ['emi', 'monthly payment']):
            if 'remaining' in question_lower or 'left' in question_lower:
                return "SELECT loan_id, tenure_months - ROUND(amount_paid/monthly_emi) as emis_remaining FROM loan_data WHERE loan_id = ?"
            else:
                return "SELECT loan_id, monthly_emi FROM loan_data WHERE loan_id = ?"
        
        elif any(word in question_lower for word in ['balance', 'outstanding']):
            return "SELECT loan_id, loan_amount - amount_paid as outstanding_balance FROM loan_data WHERE loan_id = ?"
        
        elif any(word in question_lower for word in ['topup', 'top-up', 'eligible']):
            return "SELECT loan_id, topup_eligible, status, amount_paid FROM loan_data WHERE loan_id = ?"
        
        elif any(word in question_lower for word in ['details', 'information', 'show']):
            return "SELECT * FROM loan_data WHERE loan_id = ?"
        
        else:
            return "SELECT * FROM loan_data WHERE loan_id = ? LIMIT 1"
    
    def execute_query(self, question: str, loan_id: str = None) -> Dict[str, Any]:
        """Execute query based on natural language"""
        print(f" SQL Analyst processing: '{question}'")
        
        try:
            sql_query = self.natural_language_to_sql(question)
            
            if loan_id:
                result_df = self.db.execute_safe_query(sql_query, (loan_id,))
            else:
                result_df = self.db.execute_safe_query(sql_query)
            
            response = {
                "question": question,
                "sql_query": sql_query,
                "data": result_df.to_dict('records') if not result_df.empty else [],
                "success": not result_df.empty
            }
            
            if result_df.empty:
                response["message"] = "No data found for the specified criteria"
            else:
                response["message"] = f"Found {len(result_df)} record(s)"
            
            return response
            
        except Exception as e:
            return {
                "question": question,
                "sql_query": "ERROR",
                "data": [],
                "success": False,
                "message": f"Query execution failed: {e}"
            }

# Initialize SQL Analyst
sql_analyst = SQLAnalystAgent(db_manager)


 SQL Analyst Agent initialized


In [13]:
# Cell 8: Supervisor Agent
class SupervisorAgent:
    """Supervisor Agent for orchestration and intent recognition"""
    
    def __init__(self, sql_analyst: SQLAnalystAgent, policy_guru: PolicyGuruAgent, 
                 calculator: AmortizationCalculator):
        self.sql_analyst = sql_analyst
        self.policy_guru = policy_guru
        self.calculator = calculator
        self.conversation_history = []
        print(" Supervisor Agent initialized - Ready for orchestration")
    
    def classify_intent(self, question: str) -> Dict[str, Any]:
        """Classify user intent"""
        question_lower = question.lower()
        
        intent_weights = {
            "data_query": 0,
            "policy_query": 0, 
            "calculation": 0,
            "combined": 0
        }
        
        # Data query indicators
        data_keywords = ['emi', 'balance', 'outstanding', 'remaining', 'status', 'details']
        if any(keyword in question_lower for keyword in data_keywords):
            intent_weights["data_query"] += 2
        
        # Policy query indicators
        policy_keywords = ['policy', 'eligible', 'qualify', 'rule', 'regulation', 'compliance']
        if any(keyword in question_lower for keyword in policy_keywords):
            intent_weights["policy_query"] += 2
        
        # Calculation indicators
        calc_keywords = ['prepay', 'prepayment', 'what if', 'simulate', 'calculate', 'how much']
        if any(keyword in question_lower for keyword in calc_keywords):
            intent_weights["calculation"] += 2
        
        # Determine primary intent
        primary_intent = max(intent_weights, key=intent_weights.get)
        confidence = intent_weights[primary_intent] / 2  # Normalize to 0-1 scale
        
        # Check for combined intent
        if sum(1 for weight in intent_weights.values() if weight > 0) > 1:
            primary_intent = "combined"
        
        return {
            "primary_intent": primary_intent,
            "confidence": confidence,
            "all_intents": intent_weights
        }
    
    def extract_loan_id(self, question: str) -> Optional[str]:
        """Extract loan ID from question"""
        # Simple pattern matching - in production would use more sophisticated NLP
        words = question.upper().split()
        for word in words:
            if word.startswith('LOAN') or word.startswith('L'):
                return word
        return "LOAN1001"  # Default for demo
    
    def extract_prepayment_amount(self, question: str) -> Optional[float]:
        """Extract prepayment amount from question"""
        import re
        amounts = re.findall(r'₹?\s*(\d+(?:,\d+)*(?:\.\d+)?)\s*', question)
        if amounts:
            # Convert to float (remove commas)
            return float(amounts[0].replace(',', ''))
        return None
    
    def orchestrate_query(self, question: str) -> Dict[str, Any]:
        """Orchestrate the multi-agent response"""
        print(f"\n{'='*60}")
        print(f" SUPERVISOR PROCESSING: '{question}'")
        print(f"{'='*60}")
        
        # Step 1: Intent classification
        intent_result = self.classify_intent(question)
        primary_intent = intent_result["primary_intent"]
        confidence = intent_result["confidence"]
        
        print(f" Intent: {primary_intent} (confidence: {confidence:.2f})")
        
        # Step 2: Extract parameters
        loan_id = self.extract_loan_id(question)
        prepayment_amount = self.extract_prepayment_amount(question)
        
        print(f" Parameters - Loan ID: {loan_id}, Prepayment: {prepayment_amount}")
        
        # Step 3: Route to appropriate agents
        responses = {}
        
        if primary_intent in ["data_query", "combined"]:
            print(" Routing to SQL Analyst...")
            responses["sql_analyst"] = self.sql_analyst.execute_query(question, loan_id)
        
        if primary_intent in ["policy_query", "combined"]:
            print(" Routing to Policy Guru...")
            responses["policy_guru"] = self.policy_guru.query_policy(question)
        
        if primary_intent in ["calculation", "combined"] and prepayment_amount:
            print(" Routing to What-If Calculator...")
            # Get loan data first
            loan_data_response = self.sql_analyst.execute_query("get loan details", loan_id)
            if loan_data_response["success"] and loan_data_response["data"]:
                loan_data = loan_data_response["data"][0]
                responses["calculator"] = self.calculator.simulate_prepayment(loan_data, prepayment_amount)
        
        # Step 4: Merge responses
        final_response = self.merge_responses(question, responses, primary_intent)
        
        # Store in conversation history
        self.conversation_history.append({
            "timestamp": datetime.now().isoformat(),
            "question": question,
            "response": final_response
        })
        
        return final_response
    
    def merge_responses(self, question: str, responses: Dict, intent: str) -> Dict[str, Any]:
        """Merge responses from multiple agents"""
        print(" Merging agent responses...")
        
        final_response = {
            "original_question": question,
            "intent": intent,
            "timestamp": datetime.now().isoformat(),
            "summary": "",
            "detailed_responses": responses,
            "recommendations": []
        }
        
        # Generate summary based on responses
        summary_parts = []
        
        if "sql_analyst" in responses:
            sql_data = responses["sql_analyst"]["data"]
            if sql_data:
                summary_parts.append(" Found your loan details in our system.")
        
        if "policy_guru" in responses:
            policy_answer = responses["policy_guru"]["answer"]
            summary_parts.append(f" Policy information: {policy_answer}")
        
        if "calculator" in responses:
            calc_result = responses["calculator"]
            if "prepayment_impact" in calc_result:
                impact = calc_result["prepayment_impact"]
                summary_parts.append(f" Prepayment would save ₹{impact['interest_saved']:,.2f} in interest")
        
        final_response["summary"] = " ".join(summary_parts) if summary_parts else "Please provide more specific details about your query."
        
        # Add recommendations
        if "sql_analyst" in responses and responses["sql_analyst"]["success"]:
            final_response["recommendations"].append("You can ask about EMI details, outstanding balance, or top-up eligibility.")
        
        if "policy_guru" in responses:
            final_response["recommendations"].append("For detailed policy documents, visit our customer portal.")
        
        print(" Response merging completed")
        return final_response

# Initialize Supervisor
supervisor = SupervisorAgent(sql_analyst, policy_guru, calculator)


 Supervisor Agent initialized - Ready for orchestration


In [14]:
# Cell 9: Test the Complete System
print(" TESTING LOAN NAVIGATOR AGENT SUITE")
print("=" * 60)

# Test queries
test_queries = [
    "What is my remaining EMI for loan LOAN1001?",
    "Am I eligible for top-up loan?",
    "If I prepay ₹10,000, how will my EMI change for LOAN1001?",
    "What is the policy on prepayment?",
    "Show my loan details for LOAN1001"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'➖'*30}")
    print(f"TEST {i}: {query}")
    print(f"{'➖'*30}")
    
    response = supervisor.orchestrate_query(query)
    
    print(f"\n FINAL RESPONSE SUMMARY:")
    print(f"   {response['summary']}")
    
    if response['recommendations']:
        print(f" RECOMMENDATIONS:")
        for rec in response['recommendations']:
            print(f"   - {rec}")

print(f"\n SYSTEM TESTING COMPLETED!")


 TESTING LOAN NAVIGATOR AGENT SUITE

➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖
TEST 1: What is my remaining EMI for loan LOAN1001?
➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖

 SUPERVISOR PROCESSING: 'What is my remaining EMI for loan LOAN1001?'
 Intent: data_query (confidence: 1.00)
 Parameters - Loan ID: LOAN, Prepayment: 1001.0
 Routing to SQL Analyst...
 SQL Analyst processing: 'What is my remaining EMI for loan LOAN1001?'
 Query execution failed: Execution failed on sql 'SELECT loan_id, tenure_months - ROUND(amount_paid/monthly_emi) as emis_remaining FROM loan_data WHERE loan_id = ?': no such table: loan_data
 Merging agent responses...
 Response merging completed

 FINAL RESPONSE SUMMARY:
   Please provide more specific details about your query.

➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖
TEST 2: Am I eligible for top-up loan?
➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖

 SUPERVISOR PROCESSING: 'Am I eligible for top-up loan?'
 Intent: policy_query (confidence: 1.00)
 Parameters - Loan ID: LOAN?, Prepayment: None
 Routing to Po

In [ ]:
# Cell 11: Interactive Demo
class InteractiveDemo:
    """Interactive demo for the Loan Navigator"""
    
    def __init__(self, supervisor: SupervisorAgent):
        self.supervisor = supervisor
        self.demo_loans = [
            {"loan_id": "LOAN1001", "amount": 75000, "interest": 12, "tenure": 12},
            {"loan_id": "LOAN1002", "amount": 150000, "interest": 11.5, "tenure": 24},
            {"loan_id": "LOAN1003", "amount": 50000, "interest": 13, "tenure": 6}
        ]
    
    def run_demo(self):
        """Run interactive demo"""
        print("\n" + "" * 30)
        print("   INTERACTIVE DEMO - LOAN NAVIGATOR")
        print("" * 30)
        
        print("\n Available demo loans:")
        for loan in self.demo_loans:
            print(f"   • {loan['loan_id']}: ₹{loan['amount']:,.2f} at {loan['interest']}% for {loan['tenure']} months")
        
        print("\n Try these queries or ask your own:")
        print("   - 'What is my EMI for LOAN1001?'")
        print("   - 'Can I get a top-up loan?'") 
        print("   - 'What happens if I prepay ₹10,000 for LOAN1001?'")
        print("   - 'What is your prepayment policy?'")
        print("   - Type 'exit' to quit\n")
        
        while True:
            try:
                user_input = input(" Your question: ").strip()
                
                if user_input.lower() in ['exit', 'quit', 'bye']:
                    print(" Thank you for using Loan Navigator!")
                    break
                
                if not user_input:
                    continue
                
                # Process query
                response = self.supervisor.orchestrate_query(user_input)
                
                # Display response
                print(f"\n LOAN NAVIGATOR:")
                print(f"   {response['summary']}")
                
                if response['recommendations']:
                    print(f"\n Suggestions:")
                    for rec in response['recommendations']:
                        print(f"   • {rec}")
                
                print("-" * 50)
                
            except KeyboardInterrupt:
                print("\n Demo ended by user")
                break
            except Exception as e:
                print(f" Error: {e}")

# Run interactive demo
demo = InteractiveDemo(supervisor)
demo.run_demo()




   INTERACTIVE DEMO - LOAN NAVIGATOR


 Available demo loans:
   • LOAN1001: ₹75,000.00 at 12% for 12 months
   • LOAN1002: ₹150,000.00 at 11.5% for 24 months
   • LOAN1003: ₹50,000.00 at 13% for 6 months

 Try these queries or ask your own:
   - 'What is my EMI for LOAN1001?'
   - 'Can I get a top-up loan?'
   - 'What happens if I prepay ₹10,000 for LOAN1001?'
   - 'What is your prepayment policy?'
   - Type 'exit' to quit


 SUPERVISOR PROCESSING: 'What is my remaining EMI for loan LOAN1001'
 Intent: data_query (confidence: 1.00)
 Parameters - Loan ID: LOAN, Prepayment: 1001.0
 Routing to SQL Analyst...
 SQL Analyst processing: 'What is my remaining EMI for loan LOAN1001'
 Query execution failed: Execution failed on sql 'SELECT loan_id, tenure_months - ROUND(amount_paid/monthly_emi) as emis_remaining FROM loan_data WHERE loan_id = ?': no such table: loan_data
 Merging agent responses...
 Response merging completed

 LOAN NAVIGATOR:
   Please provide more specific details about your